In [6]:
import pandas as pd
import numpy as np
import csv

print "Reading csv file."
df = pd.read_csv("ubuntu_centos_classification.csv")

Y= df['OS_type']
X = df.values[:,:-1]

for j in range(0,X.shape[1]):
    maximum = max(X[:,j]) if max(X[:,j]) !=0 else 1 
    for i in range(0,len(X)):
        X[i,j] = round(X[i,j]/maximum,3)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
print X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

cleanup_nums = {"centos":0, "ubuntu":1}
Y_train.replace(cleanup_nums,inplace=True)
Y_test.replace(cleanup_nums,inplace=True)

Reading csv file.
(21, 47) (10, 47) (21,) (10,)


In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Model for binary classification
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(47, input_dim=47, kernel_initializer='random_uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='random_uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_baseline()
model.summary()
estimator = KerasClassifier(build_fn=create_baseline, batch_size=100, verbose=0)
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
print("Train Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

results = cross_val_score(estimator, X_test, Y_test, cv=kfold)
print("Test Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 47)                2256      
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 48        
Total params: 2,304
Trainable params: 2,304
Non-trainable params: 0
_________________________________________________________________
Train Results: 95.00% (5.00%)
Test Results: 80.00% (20.00%)
